## Predicting with Kalman Filters and AR

Take the predicted state from process space of what comes out of model and use it in predictions  

Assume measurements are poor, model that by having random noise on measurements  

Assume we have a perfect model and assume need to change measurements  
Gonna operate on the lags of   
Parameters won't change  

**Take what comes out of the Kalman Gain Model => turn into an observation => use it to do predictions with AR**

In the ```kalman_step``` function we now return the predicted observation as well as the state estimate and state covariance

In [20]:
import numpy as np
def kalman_step (F,Q,R,state_covariance,H, state_estimate,observation):

    predicted_state = F @ state_estimate
    predicted_covariance = F @ state_covariance @ F.T + Q
    
    predicted_observation = H @ predicted_state # new addition
    
    prefit_residual = observation - H @ predicted_state
    prefit_covariance = H @ predicted_covariance @ H.T + R
    kalman_gain = (predicted_covariance @ np.transpose(H)) @ np.linalg.inv(prefit_covariance)
    state_estimate = predicted_state + kalman_gain @ prefit_residual
    state_covariance = (np.identity(len(state_estimate)) - kalman_gain @ H) @ state_covariance

    # return predicted_observation as well
    return (predicted_observation, state_estimate,   state_covariance)

Changes in the ```kalman_multi``` function  
- add in predictions = []
- in loop append new predicted observations to array
- return predictions

In [30]:
def kalman_multi (F,Q,R,state_covariance,H, state_estimate, observations):
    predictions = [] ##
    for i in range (len(observations)):
        (predicted_observation, state_estimate, state_covariance) = kalman_step(F,Q,R,state_covariance,H, state_estimate, observations[i])
        predictions.append(predicted_observation) ##
        # print(' state after :', i, 'is',  state_estimate)
    return predictions

### Tesla Example
now adding some random noise to it with:  
 ``` series = tsla + 0.1 * np.random.randn(len(tsla)) ```

Idea is that we have noisy measurements of the prices, trying to get from these noisy readings back to better more accurate obseravtions of the prices

In [22]:
import yfinance
import statsmodels.api as sm

tsla = np.log(yfinance.Ticker('TSLA').history(start='2015-01-01', end='2022-01-01', interval='1d').reset_index()['Close'])

series = tsla + 0.1 * np.random.randn(len(tsla))

ar_deg = 3

model = sm.tsa.AutoReg(series, lags = ar_deg, trend='n').fit()

#### AR(3): states  
A state will be a vector of lags. Length will be the order (3)  
``` state_estimate = np.array(series[ar_deg-1::-1]) ``` 
will return ``` [a3, a2, a1] ```

In [23]:
state_estimate = np.array(series[ar_deg-1::-1])
print(state_estimate)

[3.63462577 3.61893205 3.77032727]


#### AR(3): state transition
Gives a matrix the first row of which is the AR paramaters
- dot that with last three lags to get prediction for next state  

Second row is [1 0 0 ...] 
- used to be the first lag is now the second  

In general get the paramaters on top and rest are shifting one



In [24]:
F = np.vstack([model.params, np.hstack([np.identity(ar_deg - 1), np.zeros((ar_deg - 1, 1))])])
F

array([[0.38935111, 0.29965924, 0.31172904],
       [1.        , 0.        , 0.        ],
       [0.        , 1.        , 0.        ]])

#### Noise
Process noise gives a 1 (the model_cov) on the top row and the others all zeros.  
Be uncertainty for the AR paramater multiplication but not for the shifts  

Measurement Noise is a singleton, just 1 here

In [25]:
model_cov = 1.0

# Process Noise
Q = np.diag([model_cov] + [0.]*(ar_deg - 1))

observation_cov = 1.0

# Measurement Noise
R = np.array(observation_cov)

print(Q,'\n', R)

[[1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]] 
 1.0


```H = [1 0 0 0 ...]``` because to get from series of lags to a price just take top one

In [26]:
H = np.array([[1] + [0.] * (ar_deg - 1)])
H

array([[1., 0., 0.]])

In [27]:
state_covariance = np.zeros((ar_deg, ar_deg))


In [28]:
observations = np.array(series)[ar_deg: ]
# ignore the first ar_deg amount in the series
observations

array([3.87789977, 3.6520388 , 3.62414825, ..., 7.06488899, 6.99123073,
       7.13525668])

#### Predicted Observations

In [36]:
import math
# predicted_observations = kalman_multi(F, Q, R, state_covariance, H, state_estimate, observations)
predicted_observations = kalman_multi(F,Q,R,state_covariance,H,state_estimate, observations)

# create predicted values array with ar_deg nan to start series off
predicted_values = [math.nan] * ar_deg

# add predicted observations to predicted values
for state in predicted_observations: 
    predicted_values.append(state[0])

print(predicted_observations[:3], predicted_values[:6])

[array([3.6749125]), array([3.68762332]), array([3.69350618])] [nan, nan, nan, 3.674912496624447, 3.687623320746377, 3.6935061822298594]


#### Compare RMSE of models
1st model is the KF with AR

Changed the series but have not changed the model

In [37]:
def rmse(predicted, actual):
    # print(predicted, actual)1
    diffs = predicted - actual
    # print(diffs)
    return math.sqrt(np.mean(diffs ** 2))

rmse(np.concatenate(predicted_observations), series[ar_deg:])


0.11983011548369023

Second model is the straight AR(3) model

In [38]:

rmse(model.predict(), series)


0.1219359817318891

## AR with Kalman Filters Pt. 2

[Lecture Video](https://learn.london.ac.uk/mod/page/view.php?id=80173&forceview=1)
No slides for this lecture so some missed stuff might be in video

Idea behind above example is we some parameters and have perturbed measurements. 
Going to keep the paramaters but change the measurements.  

Here doing the opposite. Change the parameters and keep measurements even though they are still perturbed by noise.  

Similar in spirit to an ARMA with p=1 

Different setup:
- assume we have the readings and going to adapt the model parameters
- much more like we are used to doing
- In Kalman filter terms, a state is a vector of parameters
- H doing a lot of work, moves us from the parameters to the observable space (price) using dot product

In KF terms:
1. States are the parameters
2. Prediction stage, we won't change the state => F = Identity Matrix
- state is ``` [alpha1, alpha2, alpha3, alpha4] ```
- F is not not doing much, just identity matrix
- Q is zeros. Q normally the noise that comes with the F matrix which isn't doing anything Expect noise in the identity computation  
- Real action is in H

H
- H is transformation that takes you from states to observables
- State is the paramaters. Observable = share price
- What do we need to multipyly the parameters by to get a share price?
    - LAGS
H will be 4 lags. It will be a different vector of lags for each prediction.  
The H's will change with each step  

To implement - need to change the multi-step code  
- instead of taking a static H as input, take an array that we call Hs  
- array will have a vector of 4 lags at each position  



In [ ]:
def kalman_step_2 (F,Q,R,state_covariance,H, state_estimate,observation):

    predicted_state = F @ state_estimate
    predicted_covariance = F @ state_covariance @ F.T + Q
    
    predicted_observation = H @ predicted_state # new addition
    
    prefit_residual = observation - H @ predicted_state
    prefit_covariance = H @ predicted_covariance @ H.T + R

    kalman_gain = (predicted_covariance @ np.transpose(H)) @ np.linalg.inv(prefit_covariance)
    
    state_estimate = predicted_state + kalman_gain @ prefit_residual
    state_covariance = (np.identity(len(state_estimate)) - kalman_gain @ H) @ state_covariance

    # return predicted_observation as well
    return (predicted_observation, state_estimate,   state_covariance)

In [42]:
def kalman_multi_2 (F,Q,R,state_covariance, Hs, state_estimate, observations):
    predictions = [] ##
    for i in range (len(observations)):
        (predicted_observation, state_estimate, state_covariance) = kalman_step(F,Q,R,state_covariance, Hs[i], state_estimate, observations[i])
        predictions.append(predicted_observation) ##
        # print(' state after :', i, 'is',  state_estimate)
    return predictions

## Example 2

In [39]:
ar_deg = 4

model2 = sm.tsa.AutoReg(series, lags=ar_deg, trend='n').fit()

rmse(model2.predict(), series)

0.12034237700866157

In [40]:
F = np.identity(ar_deg)
Q = np.zeros((ar_deg, ar_deg))
observation_cov = 1.0
R = np.array ([[observation_cov]])
state_covariance = np.identity(ar_deg)

In [44]:
Hs = []
for i in range (len(series) - ar_deg):
    Hs.append(np.array([series[i : i + ar_deg]]))

In [46]:
state_estimate = np.array([0] * (ar_deg - 1) + [1])

predicted_observations = kalman_multi_2(F, Q, R, state_covariance, Hs, state_estimate, np.array(series[ar_deg:]))

Outputs shown in [lecture video](https://learn.london.ac.uk/mod/page/view.php?id=80173&forceview=1) at time 19:00 and 19:15